### 1. video input 2 audio
youtube links 2 video 2 audio

winget install ffmpeg   
restore shell

In [ ]:
#from pytube import YouTube
#yt = YouTube('https://www.youtube.com/watch?v=7fMKxYBNCfc')
#stream = yt.streams.filter(only_audio=True).first()
#stream.download('/videos/testvideo1.mp4')

#bad getaway

import yt_dlp

def download_audio(url, output_path='audio.mp3'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio', # using ffmpeg to convert the audio
            'preferredcodec': 'mp3',  # desired audio format
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

download_audio("https://www.youtube.com/watch?v=7fMKxYBNCfc")

[youtube] Extracting URL: https://www.youtube.com/watch?v=7fMKxYBNCfc
[youtube] 7fMKxYBNCfc: Downloading webpage
[youtube] 7fMKxYBNCfc: Downloading tv client config
[youtube] 7fMKxYBNCfc: Downloading player 73381ccc-main
[youtube] 7fMKxYBNCfc: Downloading tv player API JSON
[youtube] 7fMKxYBNCfc: Downloading ios player API JSON
[youtube] 7fMKxYBNCfc: Downloading m3u8 information
[info] 7fMKxYBNCfc: Downloading 1 format(s): 251
[download] audio.mp3 has already been downloaded
[download] 100% of    9.46MiB
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.orig.mp3 (pass -k to keep)


### 2. Audio 2 text

In [3]:
import whisper
model = whisper.load_model("small")
result = model.transcribe("audio.mp3")
print(result["text"])

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 76.2MiB/s]


 Ah, okay. I think I'm now ready to watch English movies without subtitles. Let's go. She's lit. She's... she's lit? Lit, lit, lit. Wasn't in the book. Let's watch another one. He's just low-key. He's just low-key? Low-key? What does it mean? Let's watch another one. Are my princely robes not on fleek? My princely robes not on fleek? On fleek? What's that? One more. Good. Jonesing for a cup of coffee. What? Jonesing? Jones? Jonesing for a cup of coffee? But these are not in my book. What are these? Hello. My name is Maddie from POC English and in this lesson I am going to teach you 15 American Slangs that you cannot learn from books. But first, what are slangs? Slangs are informal English vocabulary that you can only use in informal conversations. Now in this lesson, we are going to learn 15 very common American Slangs that you will frequently hear if you are watching movies or TV series. But before that, if you want to receive the summary of this lesson, all the 15 slangs with all the

In [9]:
print(result["text"])

 Ah, okay. I think I'm now ready to watch English movies without subtitles. Let's go. She's lit. She's... she's lit? Lit, lit, lit. Wasn't in the book. Let's watch another one. He's just low-key. He's just low-key? Low-key? What does it mean? Let's watch another one. Are my princely robes not on fleek? My princely robes not on fleek? On fleek? What's that? One more. Good. Jonesing for a cup of coffee. What? Jonesing? Jones? Jonesing for a cup of coffee? But these are not in my book. What are these? Hello. My name is Maddie from POC English and in this lesson I am going to teach you 15 American Slangs that you cannot learn from books. But first, what are slangs? Slangs are informal English vocabulary that you can only use in informal conversations. Now in this lesson, we are going to learn 15 very common American Slangs that you will frequently hear if you are watching movies or TV series. But before that, if you want to receive the summary of this lesson, all the 15 slangs with all the

### 3. Model training

https://www.kaggle.com/datasets/therohk/urban-dictionary-words-dataset?utm_source=chatgpt.com  
dataset download

In [ ]:
'''
#!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("therohk/urban-dictionary-words-dataset")

print("Path to dataset files:", path)
'''


'\n#!pip install kagglehub\nimport kagglehub\n\n# Download latest version\npath = kagglehub.dataset_download("therohk/urban-dictionary-words-dataset")\n\nprint("Path to dataset files:", path)\n'

In [ ]:
import pandas as pd

df = pd.read_csv('urbandict-word-defs.csv', on_bad_lines='skip')

df['score'] = df['up_votes'] - df['down_votes']
df_clean = df.dropna(subset=['word', 'definition'])
df_clean = df_clean.sort_values('score', ascending=False).drop_duplicates('word').head(20000)

from wordfreq import zipf_frequency

slang_dict = {
    word: definition for word, definition in zip(df_clean['word'].str.lower(), df_clean['definition'])
    if len(word) > 3 and zipf_frequency(word, 'en') < 4  # zipf freq < 4 filters out common words
}



In [ ]:
#!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.4 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/12.8 MB 8.0 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.8 MB 20.8 MB/s eta 0:00:01
     --------------------- ------------------ 6.8/12.8 MB 36.2 MB/s eta 0:00:01
     ------------------------------ -------- 10.2/12.8 MB 43.4 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 73.1 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 65.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")


patterns = [nlp.make_doc(word) for word in slang_dict.keys()]
matcher.add("SLANG", patterns)

def extract_slang_phrases(text):
    doc = nlp(text)
    matches = matcher(doc)
    found = set()
    for match_id, start, end in matches:
        phrase = doc[start:end].text.lower()
        found.add(phrase)
    return list(found)

In [16]:
text = "She totally ghosted him after the date. That party was lit!"
slangs = extract_slang_phrases(text)
print("slang:", slangs)

slang: ['ghosted']


In [27]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="gpt2-medium")
set_seed(42)

def generate_example(slang, definition):
    prompt = f"Write a  English sentence using the slang word '{slang}' to help people understand its meaning. which its meaning is: {definition}"
    output = generator(prompt, max_length=150, num_return_sequences=1, truncation=True)
    return output[0]['generated_text'].replace(prompt, "").strip()


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [29]:
for slang in slangs:
    definition = slang_dict.get(slang, "No definition found.")
    example = generate_example(slang,definition)
    print(f" Slang: {slang}\n Definition: {definition}\n Example: {example}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 Slang: ghosted
 Definition: When a person expresses love, interest, or affection for someone or something and the person they are speaking to only says 'Ditto', like Patrick Swayze in the movie Ghost.
 Example: (or similar)

I got a friend here in this town who can tell an English sentence using the slang word 'ghosted' to help people understand its meaning. which its meaning is: When a person expresses love, interest, or affection for someone or something and the person they are speaking to only says 'Ditto', like Patrick Swayze in the movie Ghost.

2.

What is the



improvment:  
应该搭建一个detect slang的模型 现在这个就是单纯的对比有没有 不太行 套个bert来试试finetune成标注模型 数据可以再好好清洗一下   
gpt2生成的例句太垃圾了 可能得换个模型 自己训练模型不太可能 没有那么大算力 

网页实时转换没想好怎么实现会比较好一点 可能最开始的时候识别完再转？